In [8]:
import requests
import torch
import torch.nn as nn
import os
from torchvision import models
from PIL import Image
from tqdm import tqdm

In [10]:
from torch.utils.data import Dataset
import requests
import torch
import torch.nn as nn
# Do install:
# conda install onnx
# conda install onnxruntime
import onnxruntime as ort
import numpy as np
import json
import io
import sys
import base64
from typing import Tuple
import pickle
import os
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset

In [11]:
os.makedirs("out/models", exist_ok=True)

Create new model

In [12]:
model = models.resnet50(weights=None)
model.fc = nn.Linear(model.fc.weight.shape[1], 10)
torch.save(model.state_dict(), "out/models/dummy_submission.pt")

### Load data

In [13]:
# transform images to tensors
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [14]:
class TaskDataset(Dataset):
    def __init__(self, transform=transform):
        self.ids = []
        self.imgs = []
        self.labels = []

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

In [15]:
dataset = TaskDataset()
dataset: TaskDataset = torch.load("Train.pt",transform)

In [20]:
# load the data
images = []
ids = []
labels = []
for id, img, label in dataset:
    img = img.convert('RGB')
    images.append(img)
    ids.append(id)
    labels.append(label)


In [21]:
imgs_tensor = [transform(img) for img in images]

In [22]:
class TensorDataset(Dataset):
    def __init__(self, ids, imgs_tensor, labels):

        self.ids = ids
        self.imgs = imgs_tensor
        self.labels = labels

        # self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        # img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

In [23]:
# create a dataset of tensors
dataset = TensorDataset(ids, imgs_tensor, labels)


### Training

Fast Gradient Sign Method (FGSM)-generate adversarial
examples with a single gradient step

� = � + � sgn(∇!� �, �, � )

Projected Gradient Descent (PGD) - multiple random restarts.

1. Train normal model

2. Add adversarial samples during training


 maximize the loss using � and �
 ���
+∈%
� � + �, �, �

3. Minimize loss on adversarial samples


epsilon 0.01 - 0.03


Adaptive attack with more accurate gradient

compute the gradients many times, usually 10, in the forward & backward
passes, accumulate them, here in the grad_noise, and at the end average them to get a more accurate
gradient.


more layers - better. e.g. 40 if millions of params



In [24]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch
import torchvision.models as models
import torch.optim as optim
import torch.nn.functional as F

In [25]:
dataset_size = len(dataset)
train_size = int(1.0 * dataset_size)  # 100% for training
test_size = dataset_size - train_size 

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [29]:
model = models.resnet50(pretrained=False)

In [30]:
# 10 classes in the last layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

In [32]:
# what device is avaialable for torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
print(device)
model = model.to(device)

cuda


In [34]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [35]:
def save_checkpoint(model, optimizer, epoch, filepath):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filepath)

In [37]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader):
            ids, inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        save_checkpoint(model, optimizer, epoch, f'resnet50/model_checkpoint_epoch_{epoch}.pth')


In [38]:
def pgd_attack(model, images, labels, eps=0.03, alpha=0.01, iters=20):
    original_images = images.clone()
    for _ in range(iters):
        images.requires_grad = True
        outputs = model(images)
        model.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        
        # # Tried adding noise to the original image before generating adversarial images
        # noise_level = 0.1  # range of noise to add to the orig. image

        # # generating uniform noise
        # noise = torch.empty_like(images).uniform_(-noise_level, noise_level)
        # # adding the noise to the original image
        # images = images + noise
        
        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - original_images, min=-eps, max=eps)
        images = torch.clamp(original_images + eta, min=0, max=1).detach_()
    return images

In [39]:
# adversarial training
def train_adv_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader):
            ids, inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            adversarial_img = pgd_attack(model, inputs, labels)
            adversarial_img, labels = adversarial_img.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(adversarial_img)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        save_checkpoint(model, optimizer, epoch, f'resnet50/model_checkpoint_adversarial_epoch_{epoch}.pth')


In [ ]:
model = model.to("cpu")

In [ ]:
#### SUBMISSION ####
torch.save(model.state_dict(), "out/models/submission_1_50_last_cpu.pt")

#### Tests ####
# (these are being ran on the eval endpoint for every submission)

allowed_models = {
    "resnet18": models.resnet18,
    "resnet34": models.resnet34,
    "resnet50": models.resnet50,
}
with open("out/models/submission_1_50_last_cpu.pt", "rb") as f:
    # try:
    #     pass
    #     #model: torch.nn.Module = allowed_models["resnet18"](weights=None)
    #     #model.fc = torch.nn.Linear(model.fc.weight.shape[1], 10)
    # except Exception as e:
    #     raise Exception(
    #         f"Invalid model class, {e=}, only {allowed_models.keys()} are allowed",
    #     )
    try:
        state_dict = torch.load(f, map_location=torch.device("cpu"))
        model.load_state_dict(state_dict, strict=True)
        model.eval()
        out = model(torch.randn(1, 3, 32, 32))
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")

    assert out.shape == (1, 10), "Invalid output shape"
    
    # Send the model to the server
response = requests.post("http://34.71.138.79:9090/robustness", files={"file": open("out/models/submission_1_50_last_cpu.pt", "rb")}, headers={"token": "75184352", "model-name": "resnet50"})

# Should be 400, the clean accuracy is too low
print(response.json())

{'detail': 'Invalid model, e=RuntimeError(\'Error(s) in loading state_dict for ResNet:\\n\\tMissing key(s) in state_dict: "layer1.0.conv3.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.downsample.0.weight", "layer1.0.downsample.1.weight", "layer1.0.downsample.1.bias", "layer1.0.downsample.1.running_mean", "layer1.0.downsample.1.running_var", "layer1.1.conv3.weight", "layer1.1.bn3.weight", "layer1.1.bn3.bias", "layer1.1.bn3.running_mean", "layer1.1.bn3.running_var", "layer1.2.conv1.weight", "layer1.2.bn1.weight", "layer1.2.bn1.bias", "layer1.2.bn1.running_mean", "layer1.2.bn1.running_var", "layer1.2.conv2.weight", "layer1.2.bn2.weight", "layer1.2.bn2.bias", "layer1.2.bn2.running_mean", "layer1.2.bn2.running_var", "layer1.2.conv3.weight", "layer1.2.bn3.weight", "layer1.2.bn3.bias", "layer1.2.bn3.running_mean", "layer1.2.bn3.running_var", "layer2.0.conv3.weight", "layer2.0.bn3.weight", "layer2.0.bn3.bias", "layer2.0.